# 优化 双模

In [ ]:
import jax
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "gpu")
# enable compilation cache
jax.config.update("jax_compilation_cache_dir", "/tmp/jax_cache")
from qutip import coherent, basis, fidelity, plot_fock_distribution, thermal_dm,tensor,fock,wigner,qfunc,ket2dm
from qutip import Qobj, qsave,qload
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import time
from math import ceil
import pandas as pd
import random

import cvxpy as cp
from scipy.sparse import csr_matrix

from efficient_bosonic_tomography.displacer import Displacer,alpha2row,Alpha2Row,Alpha2RowMultiMode, Alpha2RowWigner,Alpha2RowMultiModeWigner

#函数
# from generate_data import Displacer
from tqdm import tqdm

def generate_twomode_wignerfunc(rho,N_single,num_modes,alpha,N_compute,displacer):
    
    alpha1,alpha2 = alpha[0],alpha[1]
    # 调用 displace 方法，生成位移算符
    Da1 = displacer.displace(alpha1)
    Da2 = displacer.displace(alpha2)
    Da = jnp.kron(Da1,Da2)

    # 输出结果
    # print("位移算符 Da:")
    # print(Da)

    # 生成Parity算符
    Parity = jnp.diag(jnp.exp(1j * jnp.pi * jnp.arange(N_compute)))
    for i in range(num_modes - 1) :
        Parity0 = Parity
        Parity = jnp.kron(Parity0,Parity)

    # print(Parity)
    const = (2/jnp.pi) ** num_modes # wigner前面的系数

    # matrix = rho @ Da @ Parity @ Da.conj().T #wigner待求trace部分
    Wigner = const * jnp.trace(rho @ Da @ Parity @ Da.conj().T)
    return Wigner

if __name__ == "__main__":
    num_iterations = 7 # 循环次数
    reaserch_iterations = 1 #重复测量次数
    
    z0 = np.zeros((num_iterations,reaserch_iterations))
    z1 = np.zeros((num_iterations,reaserch_iterations))
    z2 = np.zeros((num_iterations,reaserch_iterations))
    z3 = np.zeros((num_iterations,reaserch_iterations))
    z4 = np.zeros((num_iterations,reaserch_iterations)) 
    z5 = np.zeros((num_iterations,reaserch_iterations))
    z6 = np.zeros((num_iterations,reaserch_iterations))


    for q in range(reaserch_iterations):
        for p in range(num_iterations):
            start_time1 = time.time()

            #参数表
            num_modes = 2
            limit = np.sqrt(3)
            N_single = 5 + p   # 3-10
            N_compute = 24
            num_modes = 2
            noise_level = 0.00 + 0 * 0.01  #噪声的方差大小
            # noise_level = 0.00  #噪声的方差大小
            data_point = 3000 # 单个模式的总选点数目
            batch = 200 # 一个batch的大小
            iteration = data_point//batch # 循环次数

            # 画图的参数
            # num_modes = 2
            # limit = np.sqrt(3)
            # N_single = 6 # 3-10
            # N_compute = 20 
            # num_modes = 2
            # # noise_level = 0 + p * 0.02  #噪声的方差大小
            # noise_level = 0.1  #噪声的方差大小
            # data_point = 2000 # 单个模式的总选点数目
            # batch = 200 # 一个batch的大小
            # iteration = data_point//batch # 循环次数
            key = np.random.seed(37) # 一个随机密匙，np的random里没这个，
            # np里面是一个seed决定了函数发生器的初始值,然后后面的流程一样结果也是一样的（即使多个random）

            # key1, key2 = random.split(key) # 分两个新的出来

            # binomial初态
            # 双模binomial code 
            state1 = (fock(N_compute , 0) + fock(N_compute, 4)).unit()
            state2 = fock(N_compute , 2)
            state_compute = tensor(state1,state2) + tensor(state2,state1) #binomial code
            state_compute = state_compute.unit()
            rho = ket2dm(state_compute)
            rho = rho.full()

            # binomial态的
            state1 = (fock(N_single , 0) + fock(N_single, 4)).unit()
            state2 = fock(N_single , 2)
            state_compute = tensor(state1,state2) + tensor(state2,state1) #binomial code
            state = state_compute.unit()
            rho_single = ket2dm(state)
            rho_single = rho_single.full()

            # # 两脚猫初态 
            # state1 = coherent(N_compute , 1)
            # state2 = coherent(N_compute , -1)
            # state_compute = tensor(state1,state2) + tensor(state2,state1) #binomial code
            # state_compute = state_compute.unit()
            # rho = ket2dm(state_compute)
            # rho = rho.full()

            # state1 = coherent(N_single , 0)
            # state2 = coherent(N_single , 1)
            # state_compute = tensor(state1,state2) + tensor(state2,state1) #binomial code
            # state = state_compute.unit()







            # 选点
            start_time2 = time.time()
            sample_alpha_module = np.random.uniform(-limit, limit, size=(data_point, 2)) 
            sample_alpha_module = sample_alpha_module.reshape(-1, num_modes)

            sample_alpha_angle = np.random.uniform(-np.pi, np.pi, size=(data_point, 2)) 
            sample_alpha_angle = sample_alpha_angle.reshape(-1, num_modes)

            #生成data_point个alpha
            alpha_cv_list = sample_alpha_module * np.exp(sample_alpha_angle * 1j)
            end_time2 = time.time()
            sample_new = alpha_cv_list


            # wigner计算
            # start_time3 = time.time()
            # wigner_values = np.zeros((data_point,1),dtype=np.complex128)

            # displacer = Displacer(N_compute)

            # wigner_values = []
            # wigner_func = jax.jit(jax.vmap(lambda alpha: generate_twomode_wignerfunc(rho,N_single,num_modes,
            #                                         alpha,
            #                                         N_compute,displacer)))
            # for i in tqdm(range(iteration)):

            #     # Da = jax.vmap(alpha_cv_list[i * batch : (i + 1) * batch])
            #     wigner_value = wigner_func(alpha_cv_list[i * batch : (i + 1) * batch])
            #     wigner_values.append(wigner_value)

            # # vmap的处理思路：一个数组有n个要跑的，那么就同时跑着n个量，对内存压力有点大，跑不动就分一堆batch，一个batch跑一下，最后拼起来。
            # print("wigner_values",wigner_values)
            # # # 增加噪声
            # noise = np.random.normal(0, noise_level,size=(data_point, 1))
            # wigner_values = jnp.array(wigner_values).flatten() + noise.flatten()
            # # b = jnp.clip(jnp.real(jnp.array(wigner_values)), -4/np.pi**2, 4/np.pi**2)
            # b = jnp.real(jnp.array(wigner_values))
            # b = b.reshape(-1)
            # b_new = b
            # end_time3 = time.time()

            # A的计算
            start_time4 = time.time()
            A_gen = Alpha2RowMultiModeWigner(
                            rho_h=None, N_single= N_single, num_modes=num_modes, N_compute = N_compute
                        )

            A = []  # 用来存储每个batch的结果
            # jax.jit(A_gen).trace(alpha_cv_list[ batch : 2 * batch]).lower().compile()  # 预编译以提高后续运行速度
            start_time5 = time.time()
            
            warmup_input = alpha_cv_list[0:batch]
            _ = A_gen(warmup_input)

            end_time5 = time.time() - start_time5
            for i in tqdm(range(iteration)):

                # Da = jax.vmap(alpha_cv_list[i * batch : (i + 1) * batch])
                A_pieces = A_gen(alpha_cv_list[i * batch : (i + 1) * batch])
                A.append(A_pieces)
            A = jnp.array(A)  # 将A转换为jnp数组    
            A = A.reshape(data_point,N_single ** 4)

            A_new = jax.block_until_ready(A)
            end_time4 = time.time() - end_time5

            start_time3 = time.time()
            b = A @ jnp.array(rho_single).reshape((-1,1),order = 'F')
            b = b.flatten()
            b = b + np.random.normal(0, noise_level,size=b.shape) #加噪声
            # A_new = jax.block_until_ready(b)
            end_time3 = time.time()

            # A = A_gen(alpha_cv_list) 


            # %%
            _start = time.time()
            # Define and solve the CVXPY problem.
            N = N_single ** num_modes
            X = cp.Variable((N, N), hermitian=True)
            # Y = cp.Variable(nonneg=True)
            # t = cp.Variable((1,))

            constraints = [X >> 0, cp.trace(X) == 1]

            prob = cp.Problem(cp.Minimize(cp.norm2(A @ cp.vec(X, order='F') - b)), constraints)

            prob.solve(
                solver=cp.SCS,
                # mkl=True,
                # use_indirect=True,
                # mkl=False,
                verbose=True,
                ignore_dpp=True,
                # eps_abs=1e-5,
                # eps_rel = 1e-5,
            )
            elapsed_time5 = time.time() - _start #求解时间

            print("solve time:", time.time() - _start)

            print("status:", prob.status)
            print("optimal value", prob.value)
            # print("optimal var", X.value)

            rho_reconstruct = Qobj(X.value, dims=[[N_single, N_single], [N_single, N_single]]).unit()

            rho_reconstruct = rho_reconstruct / rho_reconstruct.tr()

            # state = Qobj(state,dims=[[N_single, N_single], [N_single, N_single]])

            print("fidelity:",fidelity(rho_reconstruct, state))
            # qsave(rho_reconstruct,"density_matrix_two_modes_wigner_noise = 0.05")

            print("rho_reconstruct",rho_reconstruct)
            z0[p,q] = fidelity(rho_reconstruct, state)

            end_time1 = time.time()  # 记录结束时间
            elapsed_time1 = end_time1 - start_time1  # 计算总用时
            elapsed_time2 = end_time2 - start_time2  # 计算采样用时
            elapsed_time3 = end_time3 - start_time3  # 计算生成W函数用时
            elapsed_time4 = end_time4 - start_time4  # 计算矩阵A用时
            z1[p,q] = elapsed_time1  # 计算总用时
            z2[p,q] = elapsed_time2  # 计算采样用时
            z3[p,q] = elapsed_time3  # 计算生成W函数用时
            z4[p,q] = elapsed_time4  # 计算矩阵A用时
            z5[p,q] = elapsed_time5  # 计算cvx求解用时
            print("迭代的次数 =",p+1 )
            print("测量次数 =",q+1)
            print("N =",N_single )
            # print("phase space limit =",limit)
            print(f"总用时 Iteration {p + 1}: {elapsed_time1:.4f} seconds")
            print(f"采样所用时间 Iteration {p + 1}: {elapsed_time2:.4f} seconds")
            print(f"生成W函数时间 Iteration {p + 1}: {elapsed_time3:.4f} seconds")
            print(f"矩阵A用时 Iteration {p + 1}: {elapsed_time4:.4f} seconds")
            print(f"cvx求解用时 Iteration {p + 1}: {elapsed_time5:.4f} seconds")
            print("保真度",p+1,z0[p,q])
            z6[p,q] = N_single  #用于输出自变量



    z0_average = np.mean(z0, axis=1)
    z0_std = np.std(z0, axis=1)
    z1_average = np.mean(z1, axis=1)
    z1_std = np.std(z1, axis=1)
    z2_average = np.mean(z2, axis=1)
    z2_std = np.std(z2, axis=1)
    z3_average = np.mean(z3, axis=1)
    z3_std = np.std(z3, axis=1)
    z4_average = np.mean(z4, axis=1)
    z4_std = np.std(z4, axis=1)
    z5_average = np.mean(z5, axis=1)
    z5_std = np.std(z5, axis=1)
    z6_average = np.mean(z6, axis=1)
    z6_std = np.std(z6, axis=1)

    # 数据存储
    # 创建数据
    z = np.zeros(num_iterations)
    data = [z6_average,z0_average,z1_average,z2_average,z3_average,z4_average,z5_average,z6_std,z0_std,z1_std,z2_std,z3_std,z4_std,z6_std]
    # data_average = [z5_average,z0_average,z1_average,z2_average,z3_average,z4_average]
    # data_std = [z5_std,z0_std,z1_std,z2_std,z3_std,z4_std]

    # 转换为 DataFrame
    df = pd.DataFrame(data)

    # 将数据写入 Excel 文件
    df.to_excel('wigner_qst_convex_conic_constraint_two_mode.xlsx', index=False)

    print("数据已成功写入 wigner_qst_convex_conic_constraint_two_mode.xlsx")



# rho_loaded = qload("density_matrix_two_modes_wigner_noise = 0.4")

# print('fidelity',fidelity(rho_loaded, rho_reconstruct))








/tmp/ipykernel_3996230/764668657.py:174: UserWarning: A JAX array is being set as static! This can result in unexpected behavior and is usually a mistake to do.
  A_gen = Alpha2RowMultiModeWigner(
  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:00<00:00, 178.89it/s]

                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Dec 11 04:26:47 PM: Your problem has 625 variables, 626 constraints, and 0 parameters.
(CVXPY) Dec 11 04:26:47 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 11 04:26:47 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 11 04:26:47 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 11 04:26:47 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 11 04:26:47 PM: Compiling problem (target solver=SCS).
(C

(CVXPY) Dec 11 04:26:48 PM: Applying reduction SCS
(CVXPY) Dec 11 04:26:48 PM: Finished problem compilation (took 1.567e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Dec 11 04:26:48 PM: Invoking solver SCS  to obtain a solution.
------------------------------------------------------------------
	       SCS v3.2.9 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 3626, constraints m: 13277
cones: 	  z: primal zero / dual free vars: 1
	  q: soc vars: 12001, qsize: 3001
	  s: psd vars: 1275, ssize: 1
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e

100%|██████████| 15/15 [00:00<00:00, 180.21it/s]


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Dec 11 04:26:51 PM: Your problem has 1296 variables, 1297 constraints, and 0 parameters.
(CVXPY) Dec 11 04:26:51 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 11 04:26:51 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 11 04:26:51 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 11 04:26:51 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 11 04:26:51 PM: Compiling problem (target solver=SCS).


100%|██████████| 15/15 [00:00<00:00, 180.65it/s]


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Dec 11 04:27:03 PM: Your problem has 2401 variables, 2402 constraints, and 0 parameters.
(CVXPY) Dec 11 04:27:03 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 11 04:27:03 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 11 04:27:03 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 11 04:27:03 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 11 04:27:03 PM: Compiling problem (target solver=SCS).


100%|██████████| 15/15 [00:00<00:00, 189.21it/s]


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Dec 11 04:28:28 PM: Your problem has 4096 variables, 4097 constraints, and 0 parameters.
(CVXPY) Dec 11 04:28:28 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 11 04:28:28 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 11 04:28:28 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 11 04:28:28 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 11 04:28:28 PM: Compiling problem (target solver=SCS).


# 未优化程序,双模（baseline）

In [1]:
import jax
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "gpu")
# enable compilation cache
jax.config.update("jax_compilation_cache_dir", "/tmp/jax_cache")
from qutip import coherent, basis, fidelity, plot_fock_distribution, thermal_dm,tensor,fock,wigner,qfunc,ket2dm
from qutip import Qobj, qsave,qload
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import time
from math import ceil
import pandas as pd
import random

import cvxpy as cp
from scipy.sparse import csr_matrix

from efficient_bosonic_tomography.displacer import alpha2row,Alpha2Row,Alpha2RowMultiMode, Alpha2RowWigner,Alpha2RowMultiModeWigner

#函数
# from generate_data import Displacer
from tqdm import tqdm
import equinox as eq
import dynamiqs as dq
from typing import List


def generate_twomode_wignerfunc_old_method(rho,N_single,num_modes,alpha,N_compute,displacer):
    
    alpha1,alpha2 = alpha[0],alpha[1]
    # 调用 displace 方法，生成位移算符
    Da1 = displacer.old_method(alpha1)
    Da2 = displacer.old_method(alpha2)
    Da = jnp.kron(Da1,Da2)

    # 输出结果
    # print("位移算符 Da:")
    # print(Da)

    # 生成Parity算符
    Parity = jnp.diag(jnp.exp(1j * jnp.pi * jnp.arange(N_compute)))
    for i in range(num_modes - 1) :
        Parity0 = Parity
        Parity = jnp.kron(Parity0,Parity)

    # print(Parity)
    const = (2/jnp.pi) ** num_modes # wigner前面的系数

    # matrix = rho @ Da @ Parity @ Da.conj().T #wigner待求trace部分
    Wigner = const * jnp.trace(rho @ Da @ Parity @ Da.conj().T)
    return Wigner

class Displacer_old(eq.Module):
    evals: jnp.ndarray
    evecs: jnp.ndarray
    range: jnp.ndarray
    t_scale: jnp.ndarray
    a: jnp.ndarray

    def __init__(self, n):
        # The off-diagonal of the real-symmetric similar matrix T.
        sym = (2 * (jnp.arange(1, n) % 2) - 1) * jnp.sqrt(jnp.arange(1, n))
        # Solve the eigensystem.

        # construct a tri-diagonal matrix
        _mat = jnp.diag(sym, 1) + jnp.diag(sym, -1)

        self.evals, self.evecs = jax.scipy.linalg.eigh(_mat)
        self.range = np.arange(n)
        self.t_scale = 1j ** (self.range % 2)

        self.a = dq.destroy(n) #这个生成的数据变量是SparseDIAQArray
        self.a = jnp.asarray(np.array(self.a))

    @jax.jit
    def new_method(self, alpha):
        # Diagonal of the transformation matrix P, and apply to eigenvectors.
        r, theta = jnp.abs(alpha), jnp.angle(alpha)
        transform = self.t_scale * (jnp.exp(1j * theta)) ** -self.range
        evecs = transform[:, None] * self.evecs
        # Get the exponentiated diagonal.
        diag = jnp.exp(1j * r * self.evals)
        return jnp.conj(evecs) @ (diag[:, None] * evecs.T)

    @jax.jit
    # @eq.filter_jit
    def old_method(self, alpha_single):
        return jax.scipy.linalg.expm(
            alpha_single * self.a.conj().T - alpha_single.conj() * self.a
            # alpha_single * self.a.dag() - alpha_single.conj() * self.a
        )

class Alpha2RowMultiModeWigner_old_method(eq.Module):  #多模的A
    rho_h: jnp.ndarray | List = eq.field(static=True)
    N_single: int = eq.field(static=True)
    num_modes: int = eq.field(static=True)
    N_compute: int | None = eq.field(static=True)
    displacer: eq.Module

    def __init__(self, rho_h=None, N_single=5, num_modes=2, N_compute=None):
        self.N_single = N_single
        self.N_compute = N_compute if N_compute is not None else N_single
        self.num_modes = num_modes
        if rho_h is None:
            rho_h = [dq.basis_dm(N_compute, 0)] # 生成N维的0态的密度矩阵
            for _ in range(num_modes - 1): # 从0开始，所以-1
                rho_h.append(dq.basis_dm(N_compute, 0))
        if isinstance(rho_h, List): # 判断是否是列表
            self.rho_h = rho_h
        else:
            self.rho_h = [rho_h]
        # 将num_modes - 1个密度矩阵一个个贴起来，并把rho_h转换为列表形式。

        # self.rho_h = jnp.array(rho_h)

        self.displacer = Displacer_old(N_compute)#生成位移算符，维度为N_compute

    # @jax.jit
    def __call__(self, alpha_vec):
        def core(alpha_multi):

            Da = [self.displacer.old_method(alpha_single) for alpha_single in alpha_multi]
            # 调用了前面Displacer类中的displace函数，即生成了一个前面所说的位移算符

            Parity = jnp.diag(jnp.exp(1j * jnp.pi * jnp.arange(self.N_compute)))

            # Da_pi = Da[0]
            res = (
                (Da[0] @ Parity @ Da[0].conj().T) # 可以试试把rho_h换为Parity函数
            ) # 这个似乎是第0个密度矩阵？求第0个的投影算符
            for i, ds in enumerate(Da[1:]):# 从第i个开始到最后一个
                # Da_pi = jnp.kron(Da_pi, ds)
                res = jnp.kron(
                    res,
                    (ds @ Parity @ ds.conj().T),
                ) # 每一个做截断，然后乘出投影算符，直积起来

            # _oper = Da_pi @ self.rho_h @ Da_pi.conj().T
            AB4 = res.reshape(self.N_compute, self.N_compute, self.N_compute, self.N_compute)                
            AB4_small = AB4[:self.N_single, :self.N_single, :self.N_single, :self.N_single]             
            _oper = AB4_small.reshape(self.N_single * self.N_single, self.N_single * self.N_single)

            # _oper = res[: (self.N_single * self.N_single),: (self.N_single * self.N_single)] 

            # 矢量化 _oper
            oper_vec = jnp.array(_oper.T.flatten(order="F")) #矢量化
            
            return oper_vec
            # return res

        return jax.vmap(core)(alpha_vec) * (2/np.pi) ** self.num_modes

In [ ]:

if __name__ == "__main__":
    num_iterations = 7 # 循环次数
    reaserch_iterations = 1 #重复测量次数
    
    z0 = np.zeros((num_iterations,reaserch_iterations))
    z1 = np.zeros((num_iterations,reaserch_iterations))
    z2 = np.zeros((num_iterations,reaserch_iterations))
    z3 = np.zeros((num_iterations,reaserch_iterations))
    z4 = np.zeros((num_iterations,reaserch_iterations)) 
    z5 = np.zeros((num_iterations,reaserch_iterations))
    z6 = np.zeros((num_iterations,reaserch_iterations))


    for q in range(reaserch_iterations):
        for p in range(num_iterations):
            start_time1 = time.time()

            #参数表
            num_modes = 2
            limit = np.sqrt(3)
            N_single = 5 + p    # 3-10
            N_compute = 24
            num_modes = 2
            noise_level = 0.00 + 0 * 0.01  #噪声的方差大小
            # noise_level = 0.00  #噪声的方差大小
            data_point = 3000 # 单个模式的总选点数目
            batch = 200 # 一个batch的大小
            iteration = data_point//batch # 循环次数

            # 画图的参数
            # num_modes = 2
            # limit = np.sqrt(3)
            # N_single = 6 # 3-10
            # N_compute = 20 
            # num_modes = 2
            # # noise_level = 0 + p * 0.02  #噪声的方差大小
            # noise_level = 0.1  #噪声的方差大小
            # data_point = 2000 # 单个模式的总选点数目
            # batch = 200 # 一个batch的大小
            # iteration = data_point//batch # 循环次数
            key = np.random.seed(37) # 一个随机密匙，np的random里没这个，
            # np里面是一个seed决定了函数发生器的初始值,然后后面的流程一样结果也是一样的（即使多个random）

            # key1, key2 = random.split(key) # 分两个新的出来

            # binomial初态
            # 双模binomial code 
            state1 = (fock(N_compute , 0) + fock(N_compute, 4)).unit()
            state2 = fock(N_compute , 2)
            state_compute = tensor(state1,state2) + tensor(state2,state1) #binomial code
            state_compute = state_compute.unit()
            rho = ket2dm(state_compute)
            rho = rho.full()

            # binomial态的
            state1 = (fock(N_single , 0) + fock(N_single, 4)).unit()
            state2 = fock(N_single , 2)
            state_compute = tensor(state1,state2) + tensor(state2,state1) #binomial code
            state = state_compute.unit()
            rho_single = ket2dm(state)
            rho_single = rho_single.full()
            # # 两脚猫初态 
            # state1 = coherent(N_compute , 1)
            # state2 = coherent(N_compute , -1)
            # state_compute = tensor(state1,state2) + tensor(state2,state1) #binomial code
            # state_compute = state_compute.unit()
            # rho = ket2dm(state_compute)
            # rho = rho.full()

            # state1 = coherent(N_single , 0)
            # state2 = coherent(N_single , 1)
            # state_compute = tensor(state1,state2) + tensor(state2,state1) #binomial code
            # state = state_compute.unit()







            # 选点
            start_time2 = time.time()
            sample_alpha_module = np.random.uniform(-limit, limit, size=(data_point, 2)) 
            sample_alpha_module = sample_alpha_module.reshape(-1, num_modes)

            sample_alpha_angle = np.random.uniform(-np.pi, np.pi, size=(data_point, 2)) 
            sample_alpha_angle = sample_alpha_angle.reshape(-1, num_modes)

            #生成data_point个alpha
            alpha_cv_list = sample_alpha_module * np.exp(sample_alpha_angle * 1j)
            end_time2 = time.time()


            # wigner计算
            # start_time3 = time.time()
            # wigner_values = np.zeros((data_point,1),dtype=np.complex128)

            # displacer = Displacer_old(N_compute)

            # wigner_values = []
            # wigner_func = jax.jit(jax.vmap(lambda alpha: generate_twomode_wignerfunc_old_method(rho,N_single,num_modes,
            #                                         alpha,
            #                                         N_compute,displacer)))
            # for i in tqdm(range(iteration)):

            #     # Da = jax.vmap(alpha_cv_list[i * batch : (i + 1) * batch])
            #     wigner_value = wigner_func(alpha_cv_list[i * batch : (i + 1) * batch])
            #     wigner_values.append(wigner_value)

            # # vmap的处理思路：一个数组有n个要跑的，那么就同时跑着n个量，对内存压力有点大，跑不动就分一堆batch，一个batch跑一下，最后拼起来。

            # print("wigner_values",wigner_values)
            # # # 增加噪声
            # noise = np.random.normal(0, noise_level,size=(data_point, 1))
            # wigner_values = jnp.array(wigner_values).flatten() + noise.flatten()
            # # b = jnp.clip(jnp.real(jnp.array(wigner_values)), -4/np.pi**2, 4/np.pi**2)
            # b = jnp.real(jnp.array(wigner_values))
            # b = b.reshape(-1)
            # b_old = b
            # end_time3 = time.time()

            # A的计算
            start_time4 = time.time()
            A_gen = Alpha2RowMultiModeWigner_old_method(
                            rho_h=None, N_single= N_single, num_modes=num_modes, N_compute = N_compute
                        )
            A = []  # 用来存储每个batch的结果
            start_time5 = time.time()
            
            warmup_input = alpha_cv_list[0:batch]
            _ = A_gen(warmup_input)

            end_time5 = time.time() - start_time5

            for i in tqdm(range(iteration)):

                # Da = jax.vmap(alpha_cv_list[i * batch : (i + 1) * batch])
                A_pieces = A_gen(alpha_cv_list[i * batch : (i + 1) * batch])
                A.append(A_pieces)
            A = jnp.array(A)  # 将A转换为jnp数组    
            A = A.reshape(data_point,N_single ** 4)
            A_new = jax.block_until_ready(A)
            end_time4 = time.time() - end_time5

            start_time3 = time.time()
            b = A @ jnp.array(rho_single).reshape((-1,1),order = 'F')
            b = b.flatten()
            b = b + np.random.normal(0, noise_level,size=b.shape) #加噪声
            # A_new = jax.block_until_ready(b)
            end_time3 = time.time()

            # %%
            _start = time.time()
            # Define and solve the CVXPY problem.
            N = N_single ** num_modes
            X = cp.Variable((N, N), hermitian=True)
            # t = cp.Variable((1,))

            constraints = [X >> 0, cp.trace(X) == 1]

            prob = cp.Problem(cp.Minimize(cp.norm2(A @ cp.vec(X, order='F') - b)), constraints)

            prob.solve(
                solver=cp.SCS,
                # mkl=True,
                # use_indirect=True,
                # mkl=False,
                verbose=True,
                ignore_dpp=True,
                # eps_abs=1e-5,
                # eps_rel = 1e-5,
            )
            elapsed_time5 = time.time() - _start #求解时间

            print("solve time:", time.time() - _start)

            print("status:", prob.status)
            print("optimal value", prob.value)
            # print("optimal var", X.value)

            rho_reconstruct = Qobj(X.value, dims=[[N_single, N_single], [N_single, N_single]]).unit()

            rho_reconstruct = rho_reconstruct / rho_reconstruct.tr()

            # state = Qobj(state,dims=[[N_single, N_single], [N_single, N_single]])

            print("fidelity:",fidelity(rho_reconstruct, state))
            # qsave(rho_reconstruct,"density_matrix_two_modes_wigner_noise = 0.05")

            print("rho_reconstruct",rho_reconstruct)
            z0[p,q] = fidelity(rho_reconstruct, state)

            end_time1 = time.time()  # 记录结束时间
            elapsed_time1 = end_time1 - start_time1  # 计算总用时
            elapsed_time2 = end_time2 - start_time2  # 计算采样用时
            elapsed_time3 = end_time3 - start_time3  # 计算生成W函数用时
            elapsed_time4 = end_time4 - start_time4  # 计算矩阵A用时
            z1[p,q] = elapsed_time1  # 计算总用时
            z2[p,q] = elapsed_time2  # 计算采样用时
            z3[p,q] = elapsed_time3  # 计算生成W函数用时
            z4[p,q] = elapsed_time4  # 计算矩阵A用时
            z5[p,q] = elapsed_time5  # 计算cvx求解用时
            print("迭代的次数 =",p+1 )
            print("测量次数 =",q+1)
            print("N =",N_single )
            # print("phase space limit =",limit)
            print(f"总用时 Iteration {p + 1}: {elapsed_time1:.4f} seconds")
            print(f"采样所用时间 Iteration {p + 1}: {elapsed_time2:.4f} seconds")
            print(f"生成W函数时间 Iteration {p + 1}: {elapsed_time3:.4f} seconds")
            print(f"矩阵A用时 Iteration {p + 1}: {elapsed_time4:.4f} seconds")
            print(f"cvx求解用时 Iteration {p + 1}: {elapsed_time5:.4f} seconds")
            print("保真度",p+1,z0[p,q])
            z6[p,q] = N_single  #用于输出自变量



    z0_average = np.mean(z0, axis=1)
    z0_std = np.std(z0, axis=1)
    z1_average = np.mean(z1, axis=1)
    z1_std = np.std(z1, axis=1)
    z2_average = np.mean(z2, axis=1)
    z2_std = np.std(z2, axis=1)
    z3_average = np.mean(z3, axis=1)
    z3_std = np.std(z3, axis=1)
    z4_average = np.mean(z4, axis=1)
    z4_std = np.std(z4, axis=1)
    z5_average = np.mean(z5, axis=1)
    z5_std = np.std(z5, axis=1)
    z6_average = np.mean(z6, axis=1)
    z6_std = np.std(z6, axis=1)

    # 数据存储
    # 创建数据
    z = np.zeros(num_iterations)
    data = [z6_average,z0_average,z1_average,z2_average,z3_average,z4_average,z5_average,z6_std,z0_std,z1_std,z2_std,z3_std,z4_std,z6_std]
    # data_average = [z5_average,z0_average,z1_average,z2_average,z3_average,z4_average]
    # data_std = [z5_std,z0_std,z1_std,z2_std,z3_std,z4_std]

    # 转换为 DataFrame
    df = pd.DataFrame(data)

    # 将数据写入 Excel 文件
    df.to_excel('wigner_qst_convex_conic_constraint_two_mode_without_optimazation.xlsx', index=False)

    print("数据已成功写入 wigner_qst_convex_conic_constraint_two_mode_without_optimazation.xlsx")



# rho_loaded = qload("density_matrix_two_modes_wigner_noise = 0.4")

# print('fidelity',fidelity(rho_loaded, rho_reconstruct))








/tmp/ipykernel_4010518/310540859.py:124: UserWarning: A JAX array is being set as static! This can result in unexpected behavior and is usually a mistake to do.
  A_gen = Alpha2RowMultiModeWigner_old_method(
100%|██████████| 15/15 [00:00<00:00, 113.86it/s]

                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Dec 11 04:30:09 PM: Your problem has 625 variables, 626 constraints, and 0 parameters.
(CVXPY) Dec 11 04:30:09 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 11 04:30:09 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 11 04:30:09 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 11 04:30:09 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 11 04:30:09 PM: Compiling problem (target solver=SCS).
(C

(CVXPY) Dec 11 04:30:10 PM: Applying reduction SCS
(CVXPY) Dec 11 04:30:10 PM: Finished problem compilation (took 1.868e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Dec 11 04:30:10 PM: Invoking solver SCS  to obtain a solution.
------------------------------------------------------------------
	       SCS v3.2.9 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 3626, constraints m: 13277
cones: 	  z: primal zero / dual free vars: 1
	  q: soc vars: 12001, qsize: 3001
	  s: psd vars: 1275, ssize: 1
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e

100%|██████████| 15/15 [00:00<00:00, 113.79it/s]


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Dec 11 04:30:13 PM: Your problem has 1296 variables, 1297 constraints, and 0 parameters.
(CVXPY) Dec 11 04:30:13 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 11 04:30:13 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 11 04:30:13 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 11 04:30:13 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 11 04:30:13 PM: Compiling problem (target solver=SCS).


100%|██████████| 15/15 [00:00<00:00, 113.50it/s]


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Dec 11 04:30:29 PM: Your problem has 2401 variables, 2402 constraints, and 0 parameters.
(CVXPY) Dec 11 04:30:29 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 11 04:30:29 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 11 04:30:29 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 11 04:30:29 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 11 04:30:29 PM: Compiling problem (target solver=SCS).


100%|██████████| 15/15 [00:00<00:00, 113.16it/s]


                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Dec 11 04:31:56 PM: Your problem has 4096 variables, 4097 constraints, and 0 parameters.
(CVXPY) Dec 11 04:31:56 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 11 04:31:56 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 11 04:31:56 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 11 04:31:56 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 11 04:31:56 PM: Compiling problem (target solver=SCS).


In [33]:
sample1 - sample2

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]], shape=(3000, 2))

In [30]:
sample2 = alpha_cv_list

In [34]:
w2 = wigner_values

In [35]:
w1 - np.array(w2)

Array([[ 0.25117635-1.48596298e-16j,  0.14395463-1.57794160e-16j,
         0.10449925-7.93494761e-17j, ..., -0.24276784+1.06034355e-16j,
         0.14335496-7.16374766e-17j,  0.02576918-1.20284725e-17j],
       [ 0.23619197-1.33169461e-16j,  0.09788134-1.12392467e-16j,
        -0.28409244+1.55967520e-16j, ..., -0.18631588+8.89586569e-17j,
         0.11472439-5.94271857e-17j,  0.07320923-7.89958182e-17j],
       [ 0.27257271-1.32605349e-16j,  0.06415856-4.09483753e-17j,
         0.09594197-4.46531739e-17j, ..., -0.07007025+2.17729795e-17j,
         0.19328868-1.26835491e-16j,  0.09510382-2.23727236e-16j],
       ...,
       [ 0.26487011-1.45581271e-16j, -0.02474289-4.02503312e-18j,
         0.05130808+1.00952055e-16j, ..., -0.03740371+2.46395754e-17j,
         0.08870587-5.14957069e-17j,  0.06867871-1.59897173e-16j],
       [ 0.15594431-8.99964686e-17j,  0.00781139+6.93005533e-18j,
         0.08372351-1.43283851e-17j, ...,  0.33502835-1.02764508e-15j,
         0.34164227-4.99262126e-16j

In [14]:
A_new-A_old

TypeError: sub got incompatible shapes for broadcasting: (3000, 4096), (3000, 20736).

In [30]:
alpha_cv_list

array([[ 0.32261545, -0.00773541],
       [-0.42189134,  0.04451962],
       [ 7.0651752 ,  1.04242789],
       ...,
       [ 0.02152629, -0.03204469],
       [-0.69027089,  2.42621158],
       [-0.76595351,  0.1074859 ]], shape=(2000, 2))

生成四维网格

In [41]:
alpha_multi =np.array( [0.1+0.2j, -0.3+0.5j] )
Da = [displacer.old_method(alpha_single) for alpha_single in alpha_multi]

# Da = displacer.old_method(alpha_single)
# 调用了前面Displacer类中的displace函数，即生成了一个前面所说的位移算符

Parity = jnp.diag(jnp.exp(1j * jnp.pi * jnp.arange(N_compute)))

# Da_pi = Da[0]
res = (
    (Da[0] @ Parity @ Da[0].conj().T)[: N_single,: N_single] # 可以试试把rho_h换为Parity函数
) # 这个似乎是第0个密度矩阵？求第0个的投影算符
for i, ds in enumerate(Da[1:]):# 从第i个开始到最后一个
    # Da_pi = jnp.kron(Da_pi, ds)
    res = jnp.kron(
        res,
        (ds @ Parity @ ds.conj().T)[: N_single,: N_single],
    ) # 每一个做截断，然后乘出投影算符，直积起来

# _oper = Da_pi @ self.rho_h @ Da_pi.conj().T
_oper = res[: N_single,: N_single] 

# 矢量化 _oper
oper_vec = jnp.array(_oper.T.flatten(order="F")) #矢量化
    
    # return oper_vec
    # return res

# return jax.vmap(core)(alpha_vec) * (2/np.pi) ** self.num_modes

In [38]:
(ds @ Parity @ ds.conj().T).shape

(20, 20)

In [17]:
A = []  # 用来存储每个batch的结果
for i in tqdm(range(iteration)):

    # Da = jax.vmap(alpha_cv_list[i * batch : (i + 1) * batch])
    A_pieces = A_gen(alpha_cv_list[i * batch : (i + 1) * batch])
    A.append(A_pieces)            


100%|██████████| 10/10 [00:00<00:00, 54.97it/s]


In [18]:
A = jnp.array(A)  

In [22]:
A = A.reshape(2000,625)
A.shape

(2000, 625)

  0%|          | 0/15 [00:00<?, ?it/s]

AttributeError: 'Partial' object has no attribute 'displace'